# 🌈⏰🌊 time-series spectra 

This example notebook provides quick visualizations to characterize a set of time-series stellar spectra for a transiting exoplanet, using the [`chromatic` toolkit](https://zkbt.github.io/chromatic/). This notebook can be run before you have fit any models to the data, requiring only the data themselves and a few details about the planet. To access this notebook, you might want to:
- Download it from [GitHub](https://github.com/zkbt/chromatic/blob/main/docs/example-timeseries-spectra.ipynb) (click 'Raw' to download the `.ipynb` file) and run on your own computer.
- Open in [Google Colab](https://colab.research.google.com/github/zkbt/chromatic/blob/main/docs/example-timeseries-spectra.ipynb), load your data into that interactive session, and run everything online.

You should be able to modify just code cells in the **💻 Load the Data + 🪐 Describe the Planet** section, and then run the entire notebook to automatically generate visualizations.

## 💾 Make Sure `chromatic` is Installed

If you don't already have `chromatic` installed, run the following command. 

In [ ]:
!pip install chromatic-lightcurves --upgrade --quiet

Once `chromatic` is installed, you should have access to all the tools you need!

In [ ]:
from chromatic import *

In [ ]:
version()

## 🧑‍💻 Load the Data + 🪐 Describe the Planet

Let's load the data into a `chromatic` 🌈 object. If you encounter errors loading your dataset, try specifying the file format with `read_rainbow(filename, format=...)` as described in the [Reading/Writing a 🌈](https://zkbt.github.io/chromatic/io/). Let's also associated a title with this object, which will automatically appear in most plots.

In [ ]:
filename = "example-datasets/stsci/jw02734*x1dints.fits"
rainbow = read_rainbow(filename)
rainbow.title = "WASP-96b | NIRISS | x1dints"

Let's define a few basic parameters describing the planet. These will help make the plots easier to interpret and be used to mask out the transit for noise characterization. If you need a period, epoch, and duration for your transit, you might find them in the [NASA Exoplanet Archive](https://exoplanetarchive.ipac.caltech.edu/).

In [ ]:
period = 3.4252577 * u.day
t0 = 2459111.30170 * u.day
duration = 2.4264 * u.hour * 1.1

Let's set some very basic plotting defaults. These will try to sync up the vertical limits of light curve plots with the color maps of 2D flux maps.

In [ ]:
vmin = 0.98
vmax = 1.005
ylim = [vmin, vmax]

## 🎨 Make Basic Visualizations

Let's make some plots to get an overall sense for the dataset. First of all, let's normalize by dividing through by the median stellar spectrum and display a 2D map of the flux. 

In [ ]:
normalized = rainbow.normalize()

In [ ]:
normalized.pcolormesh(vmin=vmin, vmax=vmax, filename="unbinned-2D-flux.png");

Let's trim any bad wavelengths off the edges and phase-fold these data to the planet's known period and time of mid-transit. This latter step is just to make the times easier to interpret.

In [ ]:
tidied = normalized.trim().fold(period=period, t0=t0)

In [ ]:
tidied.pcolormesh(vmin=vmin, vmax=vmax, filename="trimmed-2D-flux.png");

Now let's bin to a fixed wavelength resolution $R=\lambda/\Delta\lambda$ and cadence $dt$. Be averaging together multiple wavelengths and/or times, we'll decrease the noise (at the cost of worse resolution); some features may become more apparent with stronger binning, as long as the bins aren't so large to smooth features away. We'll normalize again after binning, once the noise has been averaged down a little bit.

In [ ]:
binned = tidied.bin(R=100, dt=2 * u.minute).normalize()

In [ ]:
binned.pcolormesh(vmin=vmin, vmax=vmax, filename="binned-2D-flux.png");

If we want to see how these flux maps line up to light curves, we can use the interactive exploration tool to drag and select different wavelength ranges, and plot the (unweighted) average light curves for them. Being able to select particular wavelength ranges allows us to zoom in on features of interest or trends with wavelength.

In [ ]:
binned.imshow_interact(ylim=ylim, filename="interactive-2D-flux.html")

Let's make an animation the flips through wavelength bins. Animations can be useful way to to let your eyes take in a lot of light curves in quick succession, to see how trends or noise sources change with wavelength.

In [ ]:
binned.animate_lightcurves(ylim=ylim, filename="animated-lightcurves.gif")

Let's look at the weighted average light curve, integrating together all the available wavelengths. The binned uncertainties should be smaller than any individual wavelength bin, so this should hopefully be a very precise light curve!

In [ ]:
binned.plot_average_lightcurve(ylim=ylim, filename="integrated-lightcurve.png")

Finally, let's bin the data to very low spectral resolution, and then plot a stack of light curves for the separate wavelengths. 

In [ ]:
really_binned = binned.bin(R=5)

In [ ]:
really_binned.plot(spacing=(vmax - vmin) / 2, filename="stack-of-lightcurves.png");

## 🌫 Characterize the Noise

After looking closely at the transit in the data, it can be useful to filter or mask out the transit. By creating a dataset that we expect to be mostly flat, we can characterize the noise properties. First, let's simply mask out the transit and look only at the data before and after it.

In [ ]:
out_of_transit = (
    rainbow.trim().mask_transit(period=period, t0=t0, duration=duration).normalize()
)

In [ ]:
out_of_transit.pcolormesh(
    vmin=vmin, vmax=vmax, filename="trimmed-2D-flux-with-transit-removed.png"
);

Because unbinned data have the small wavelength and time intervals, and therefore contain the relatively few photons per bin, their expected uncertainties will be large. As we bin to larger wavelength intervals (lower resolution) and longer time intervals, the expected uncertainties will decrease, revealing increasingly subtle features. 

In [ ]:
resolutions = [None, 30, 10, 3]
if rainbow.ntime > 1e4:
    dt = 1 * u.minute
else:
    dt = None

binned_rainbows = {}
N = len(resolutions)
fi, ax = plt.subplots(N, 1, sharex=True, figsize=(8, N * 3))
for i, R in enumerate(resolutions):
    binned_rainbows[R] = out_of_transit.bin(R=R, dt=dt).normalize()
    binned_rainbows[R].pcolormesh(ax=ax[i])
    plt.title(f"R={R}, dt={binned_rainbows[R].dt:.2} | {ax[i].get_title()}")
rainbow.savefig("binned-2D-flux-no-transit.png")

If the data were perfect and simple, we would expect the scatter in the out-of-transit flux to match the expected uncertainties for each wavelength. Let's compare the expected and measured scatters, after binning down in wavelength and time. If the measured scatter does not decrease as expected when averaging bins together, there is probably some systematic noise that is correlated across wavelength and/or time that should be addressed.

In [ ]:
fi, ax = plt.subplots(
    N, 2, sharey=True, figsize=(8, N * 3), gridspec_kw=dict(width_ratios=[2, 1])
)
for i, R in enumerate(binned_rainbows):
    binned_rainbows[R].plot_noise_comparison(ax=ax[i, 0], method="MAD")
    plt.title(
        f"R={R}, dt={binned_rainbows[R].dt:.2} | with transit removed\n{ax[i,0].get_title()}"
    )
    binned_rainbows[R].plot_noise_comparison_in_bins(ax=ax[i, 1], method="MAD")
rainbow.savefig("noise-comparison-no-transit.png")

It's possible that excess scatter might be the result of very smooth trends in time. Let's repeat the above analysis of the dataset binned to different wavelength and time intervals, but also remove a smooth quadratic trend in time, which could imagine might be either instrumental or astrophysical. 

In [ ]:
fi, ax = plt.subplots(N, 1, sharex=True, figsize=(8, N * 3))
for i, R in enumerate(binned_rainbows):
    with_trend_removed = binned_rainbows[R].remove_trends(method="polyfit", deg=2)
    with_trend_removed.pcolormesh(ax=ax[i])
    plt.title(f"R={R}, dt={binned_rainbows[R].dt:.2} | {ax[i].get_title()}")
rainbow.savefig("binned-2D-flux-no-transit-no-trends.png")

In [ ]:
fi, ax = plt.subplots(
    N, 2, sharey=True, figsize=(8, N * 3), gridspec_kw=dict(width_ratios=[2, 1])
)
for i, R in enumerate(binned_rainbows):
    with_trend_removed = binned_rainbows[R].remove_trends(method="polyfit", deg=2)
    with_trend_removed.plot_noise_comparison(ax=ax[i, 0], method="MAD")
    plt.title(
        f"R={R}, dt={binned_rainbows[R].dt:.2} | with transit + trends removed\n{ax[i,0].get_title()}"
    )
    with_trend_removed.plot_noise_comparison_in_bins(ax=ax[i, 1], method="MAD")
rainbow.savefig("noise-comparison-no-transit-no-trends.png")

That's it! Hopefully these automated visualizations can serve as a useful starting place for understanding the dataset you're working with and for comparing to other analyses. 



## 🗺 Explore Further! 

You can explore other options to visualize or work with your 🌈 data by using the built-in `.help()` method. On any `Rainbow` object, run this to list available options.

In [ ]:
rainbow.help()